In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

import sys
sys.path.append('../')

from src.bet_sizing import SimultaneousKelly

In [2]:
train_df = pd.read_csv("../data/processed/train.csv").drop(columns=["BOUT_ID", "EVENT_ID", "DATE", "BOUT_ORDINAL"])
test_df = pd.read_csv("../data/processed/test.csv").drop(columns=["BOUT_ID", "EVENT_ID", "DATE", "BOUT_ORDINAL"])

In [4]:
X_train = train_df.drop(columns=["RED_WIN"])
y_train = train_df["RED_WIN"]
X_test = test_df.drop(columns=["RED_WIN"])
y_test = test_df["RED_WIN"]

scalar = StandardScaler()
X_train_scaled = scalar.fit_transform(X_train)
X_test_scaled = scalar.transform(X_test)

clf = RandomForestClassifier(n_estimators=200, criterion="gini")
clf.fit(X_train_scaled, y_train)

RandomForestClassifier(n_estimators=200)

In [5]:
probs = clf.predict_proba(X_test_scaled)
red_probs, blue_probs = probs[:, 1], probs[:, 0]

In [6]:
odds = pd.read_csv("../data/processed/backtest_odds.csv")
odds["RED_MODEL_PROBS"] = red_probs
odds["BLUE_MODEL_PROBS"] = blue_probs
odds

,BOUT_ID,EVENT_ID,DATE,BOUT_ORDINAL,RED_FIGHTER_ODDS,BLUE_FIGHTER_ODDS,RED_WIN,RED_MODEL_PROBS,BLUE_MODEL_PROBS
0,3a8c0158d910546d,f5585e675af7afd4,2022-01-15,0,1.469937,2.743333,1.0,0.560,0.440
1,7c2a02b5ea169b54,f5585e675af7afd4,2022-01-15,1,1.261991,3.975000,1.0,0.645,0.355
2,3738135e4c9d8273,f5585e675af7afd4,2022-01-15,2,2.028730,1.796695,1.0,0.595,0.405
3,f9a4e625dffc30d4,f5585e675af7afd4,2022-01-15,6,1.462680,2.793333,1.0,0.665,0.335
4,8d89f3b8798f444d,f5585e675af7afd4,2022-01-15,7,1.629805,2.316667,1.0,0.485,0.515
...,...,...,...,...,...,...,...,...,...
910,2f414494f2b6f426,c398235fcaf8d71d,2024-03-16,8,2.920000,1.414494,0.0,0.460,0.540
911,acc7392d48cc09a0,c398235fcaf8d71d,2024-03-16,9,2.671429,1.480447,1.0,0.630,0.370
912,35f9beb94b4ef9e2,c398235fcaf8d71d,2024-03-16,10,5.448571,1.153292,1.0,0.245,0.755
913,a7fd9da657bbce8e,c398235fcaf8d71d,2024-03-16,11,1.466121,2.741429,NaN,0.680,0.320


In [7]:
event_ids = odds["EVENT_ID"].unique()   # preserves order of appearance
event_ids

array(['f5585e675af7afd4', '335ad945324c3a2e', '883922e5cd6d8473',
       'cbb682f5fcc44bfc', '77940e45bc86208e', 'f6a7b02c42e9fc5b',
       '75e0d3cc2262b732', '1f9344211ca7fd60', '1d00756835ca67c9',
       '1fac46d466abd5b8', '3a97fda0de6f1fa4', '8753e125f4499816',
       '236a37d96d476164', '6bf5a7d3e081ade2', '253d3f9e97ca149a',
       '60ba33c3f555a7a3', 'a6c32678efd4f434', '57591bbf1623574e',
       '3a24769a4855040a', 'a0a680fe2f6cc8e6', 'eb42d4febfafefd1',
       '4a9e305633f3ef47', '31da66df48c0c1a0', '8fd76e1b49c00ae2',
       '319c15b8aac5bfde', 'b0a6124751a56bc4', '8f6a18831a120817',
       'a23e63184c65f5b8', '4f853e98886283cf', '00a905a4a4a2b071',
       '93bf96be327fcd98', '8d477c3fbe001f9d', '4f670b7972fa0a2e',
       '49c29e57d4e2be6f', '885e7f70dcac0007', 'be5aab761c40ef35',
       '756f45905fb20cb5', 'b3b6e80b7d5f8f0d', '012fc7cd0779c09a',
       'b23388ff8ac6637b', 'f65a0eb902f9476b', '56ec58954158966a',
       '2e2cdb6e9eb84bb9', '5717efc6f271cd52', 'd26394fc0e8e88